# Build a simple LLM application with chat models and prompt templates

In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

- Using [language models](/docs/concepts/chat_models)

- Using [prompt templates](/docs/concepts/prompt_templates)

- Debugging and tracing your application using [LangSmith](https://docs.smith.langchain.com/)

Let's dive in!

## Setup

### Installation

To install LangChain run:

```{=mdx}
import Npm2Yarn from '@theme/Npm2Yarn';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Npm2Yarn>
  langchain @langchain/core
</Npm2Yarn>
```


For more details, see our [Installation guide](/docs/how_to/installation/).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."

# Reduce tracing latency if you are not in a serverless environment
# export LANGCHAIN_CALLBACKS_BACKGROUND=true
```

## Using Language Models

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably. For details on getting started with a specific model, refer to [supported integrations](/docs/integrations/chat/).

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`{ model: "gpt-4" }`} />
```

In [2]:
// @lc-docs-hide-cell
import { ChatOpenAI } from '@langchain/openai';

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
})

Let's first use the model directly. [ChatModels](/docs/concepts/chat_models) are instances of LangChain [Runnables](/docs/concepts/runnables/), which means they expose a standard interface for interacting with them. To simply call the model, we can pass in a list of [messages](/docs/concepts/messages/) to the `.invoke` method.

In [3]:
import { HumanMessage, SystemMessage } from "@langchain/core/messages"

const messages = [
  new SystemMessage("Translate the following from English into Italian"),
  new HumanMessage("hi!"),
];

await model.invoke(messages)

AIMessage {
  "id": "chatcmpl-AekSfJkg3QIOsk42BH6Qom4Gt159j",
  "content": "Ciao!",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 20,
      "completionTokens": 3,
      "totalTokens": 23
    },
    "finish_reason": "stop",
    "usage": {
      "prompt_tokens": 20,
      "completion_tokens": 3,
      "total_tokens": 23,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_6fc10e10eb"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 3,
    "input_tokens": 20,
    "total_tokens": 23,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
    "output_token_details": {
      "audio": 0,
      "reasoning": 0
    }
  }


```{=mdx}
:::tip

If we've enabled LangSmith, we can see that this run is logged to LangSmith, and can see the [LangSmith trace](https://smith.langchain.com/public/45f1a650-38fb-41e1-9b61-becc0684f2ce/r). The LangSmith trace reports [token](/docs/concepts/tokens/) usage information, latency, [standard model parameters](/docs/concepts/chat_models/#standard-parameters) (such as temperature), and other information.

:::
```

Note that ChatModels receive [message](/docs/concepts/messages/) objects as input and generate message objects as output. In addition to text content, message objects convey conversational [roles](/docs/concepts/messages/#role) and hold important data, such as [tool calls](/docs/concepts/tool_calling/) and token usage counts.

LangChain also supports chat model inputs via strings or [OpenAI format](/docs/concepts/messages/#openai-format). The following are equivalent:

```javascript
await model.invoke("Hello")

await model.invoke([ {role: "user", content: "Hello" }])

await model.invoke([new HumanMessage("hi!")])
```

### Streaming

Because chat models are [Runnables](/docs/concepts/runnables/), they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:

In [5]:
const stream = await model.stream(messages);

const chunks = [];
for await (const chunk of stream) {
  chunks.push(chunk);
  console.log(`${chunk.content}|`);
}

|
C|
iao|
!|
|
|


## Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

[Prompt templates](/docs/concepts/prompt_templates/) are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model. 

Let's create a prompt template here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [6]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

First, let's create a string that we will format to be the system message:

In [7]:
const systemTemplate = "Translate the following from English into {language}"

Next, we can create the PromptTemplate. This will be a combination of the `systemTemplate` as well as a simpler template for where to put the text

In [8]:
const promptTemplate = ChatPromptTemplate.fromMessages(
  [
    ["system", systemTemplate],
    ["user", "{text}"]
  ]
)

Note that `ChatPromptTemplate` supports multiple [message roles](/docs/concepts/messages/#role) in a single template. We format the `language` parameter into the system message, and the user `text` into a user message.

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [9]:
const promptValue = await promptTemplate.invoke({ language: "italian", text: "hi!" })

promptValue

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      SystemMessage {
        "content": "Translate the following from English into italian",
        "additional_kwargs": {},
        "response_metadata": {}
      },
      HumanMessage {
        "content": "hi!",
        "additional_kwargs": {},
        "response_metadata": {}
      }
    ]
  },
  lc_namespace: [ 'langchain_core', 'prompt_values' ],
  messages: [
    SystemMessage {
      "content": "Translate the following from English into italian",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    HumanMessage {
      "content": "hi!",
      "additional_kwargs": {},
      "response_metadata": {}
    }
  ]
}


We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [10]:
promptValue.toChatMessages()

[
  SystemMessage {
    "content": "Translate the following from English into italian",
    "additional_kwargs": {},
    "response_metadata": {}
  },
  HumanMessage {
    "content": "hi!",
    "additional_kwargs": {},
    "response_metadata": {}
  }
]


Finally, we can invoke the chat model on the formatted prompt:

In [11]:
const response = await model.invoke(promptValue)
console.log(`${response.content}`)

Ciao!


If we take a look at the LangSmith trace, we can see all three components show up in the [LangSmith trace](https://smith.langchain.com/public/6529d912-8564-4686-8df8-999c427621a7/r).

## Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to create a prompt template, and how to get great observability into applications you create with LangSmith.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](/docs/concepts).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

- [Chat models](/docs/how_to/#chat-models)
- [Prompt templates](/docs/how_to/#prompt-templates)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)